In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [193]:
DATA_FOLDER = "../Data/"

imdb_basics_raw = pd.read_csv(
    DATA_FOLDER + "IMDb/title.basics.tsv.gz",
    sep="\t",
    compression="gzip",
)
imdb_ratings_raw = pd.read_csv(
    DATA_FOLDER + "IMDb/title.ratings.tsv.gz",
    sep="\t",
    compression="gzip",
)
movies_raw = pd.read_csv(DATA_FOLDER + "MovieSummaries/movie.metadata.tsv", sep="\t")

In [194]:
movies_raw.columns = [
    "wikiID",
    "fbID",
    "name",
    "releaseDate",
    "boxOffice",
    "runtime",
    "languages",
    "countries",
    "genres",
]

imdb_basics_raw.rename(
    columns={
        "primaryTitle": "name",
        "startYear": "releaseYear",
        "runtimeMinutes": "runtime",
    },
    inplace=True
)

In [195]:
movies = movies_raw.copy()
imdb_basics = imdb_basics_raw.copy()
imdb_ratings = imdb_ratings_raw.copy()

In [196]:
# Merge both imdb datasets
imdb = pd.merge(imdb_basics, imdb_ratings, on='tconst', how='left')

In [197]:
imdb["releaseYear"] = pd.to_numeric(imdb["releaseYear"], errors="coerce")

movies["releaseMonth"] = pd.to_datetime(
    movies["releaseDate"],
    errors="coerce",
).dt.month

movies["releaseYear"] = pd.to_datetime(
    movies["releaseDate"], errors="coerce", format="mixed"
).dt.year

movies.drop(columns=["releaseDate"], inplace=True)

In [198]:
movies.drop_duplicates(subset=["name", "releaseMonth", "releaseYear"], inplace=True)
imdb.drop_duplicates(subset=["name", "releaseYear"], inplace=True)

In [199]:
imdb.runtime = pd.to_numeric(imdb.runtime, errors="coerce")

In [200]:
movies_imdb = pd.merge(movies, imdb, how="left", on=["name", "releaseYear", "runtime"], suffixes=["Cmu", "Imdb"])

In [201]:
# Function to extract genres from the string representation of dictionary-like data
def parse_dict(s):
    try:
        genres_dict = eval(s.replace('null', 'None'))
        return list(genres_dict.values())
    except:
        return None


movies_imdb['languages'] = movies_imdb['languages'].apply(parse_dict)
movies_imdb['countries'] = movies_imdb['countries'].apply(parse_dict)
movies_imdb['genresCmu'] = movies_imdb['genresCmu'].apply(parse_dict)

movies_imdb['genresImdb'] = movies_imdb['genresImdb'].apply(
    lambda x: x.split(',') if not isinstance(x, float) else np.nan)

In [202]:
print(movies.shape)
print(movies_imdb.shape)
movies_imdb.head(10)

(81630, 10)
(81630, 18)


,wikiID,fbID,name,boxOffice,runtime,languages,countries,genresCmu,releaseMonth,releaseYear,tconst,titleType,originalTitle,isAdult,endYear,genresImdb,averageRating,numVotes
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",2.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28463795,/m/0crgdbh,Brun bitter,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",NaN,1988.0,tt0094806,movie,Brun bitter,0,\N,"[Crime, Drama]",5.6,40.0
2,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",NaN,1987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,[German Language],[Germany],[Drama],NaN,1983.0,tt0083949,movie,Die flambierte Frau,0,\N,[Drama],6.0,621.0
4,13696889,/m/03cfc81,The Gangsters,NaN,35.0,"[Silent film, English Language]",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi...",5.0,1913.0,tt0002894,short,The Gangsters,0,\N,"[Comedy, Short]",6.8,16.0
5,18998739,/m/04jcqvw,The Sorcerer's Apprentice,NaN,86.0,[English Language],[South Africa],"[Family Film, Fantasy, Adventure, World cinema]",NaN,2002.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10408933,/m/02qc0j7,Alexander's Ragtime Band,3600000.0,106.0,[English Language],[United States of America],"[Musical, Comedy, Black-and-white]",8.0,1938.0,tt0029852,movie,Alexander's Ragtime Band,0,\N,"[Drama, Music, Musical]",6.8,2264.0
7,9997961,/m/06_y2j7,Contigo y aquí,NaN,NaN,[Spanish Language],[Argentina],"[Musical, Drama, Comedy]",NaN,1974.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2345652,/m/075f66,City of the Dead,NaN,76.0,[English Language],[United Kingdom],"[Horror, Supernatural]",NaN,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,175026,/m/017n1p,Sarah and Son,NaN,86.0,[English Language],[United States of America],"[Drama, Black-and-white]",NaN,1930.0,tt0021335,movie,Sarah and Son,0,\N,"[Drama, Romance]",5.4,298.0


In [203]:
characters_raw = pd.read_csv(DATA_FOLDER + "MovieSummaries/character.metadata.tsv", sep='\t')
oscars_raw = pd.read_csv(DATA_FOLDER + "Oscars/the_oscar_award.csv", sep=',')

In [204]:
characters_raw.columns = [
    "wikiID",
    "fbID",
    "releaseDate",
    "characterName",
    "DOB",
    "gender",
    "height",
    "ethnicity",
    "name",
    "ageAtMovieRelease",
    "fbCharacterMap",
    "fbCharacterID",
    "fbActorID",
]

In [205]:
oscars_raw = oscars_raw.drop(columns=["ceremony", "year_ceremony", "film"])
oscars_raw.rename(columns={"year_film": "releaseYear", "category": "oscarCategory", "winner": "oscarWinner"},
                  inplace=True)

In [206]:
characters = characters_raw.copy()
oscars = oscars_raw.copy()

In [207]:
characters["releaseYear"] = pd.to_datetime(
    characters["releaseDate"],
    errors="coerce",
    yearfirst=True,
).dt.year
characters["releaseMonth"] = pd.to_datetime(
    characters["releaseDate"],
    errors="coerce",
    yearfirst=True,
).dt.month
characters.drop(columns=["releaseDate"], inplace=True)

In [220]:
characters_oscars = pd.merge(characters, oscars, on=["name", 'releaseYear'], how="left")
print(characters_oscars.shape)
characters_oscars.head(10)

(450713, 16)


,wikiID,fbID,characterName,DOB,gender,height,ethnicity,name,ageAtMovieRelease,fbCharacterMap,fbCharacterID,fbActorID,releaseYear,releaseMonth,oscarCategory,oscarWinner
0,975900,/m/03vyhn,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4,2001.0,8.0,NaN,NaN
1,975900,/m/03vyhn,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,2001.0,8.0,NaN,NaN
2,975900,/m/03vyhn,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc,2001.0,8.0,NaN,NaN
3,975900,/m/03vyhn,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,2001.0,8.0,NaN,NaN
4,975900,/m/03vyhn,Commander Helena Braddock,1949-05-26,F,1.727,/m/0x67,Pam Grier,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,2001.0,8.0,NaN,NaN
5,975900,/m/03vyhn,Whitlock,1945-08-02,F,1.753,NaN,Joanna Cassidy,56.0,/m/02vd6kw,/m/0bgchmx,/m/06lj1m,2001.0,8.0,NaN,NaN
6,975900,/m/03vyhn,Big Daddy Mars,NaN,M,NaN,NaN,Richard Cetrone,NaN,/m/0bgchsy,/m/0bgcht0,/m/0bgcht7,2001.0,8.0,NaN,NaN
7,975900,/m/03vyhn,Michael Descanso,1971-03-20,M,1.892,NaN,Liam Waite,30.0,/m/03jqhb0,/m/0bgchs4,/m/0ks8b0,2001.0,8.0,NaN,NaN
8,975900,/m/03vyhn,Uno,NaN,M,NaN,NaN,Duane Davis,NaN,/m/0bgchtj,/m/0bgchtm,/m/03nrwdy,2001.0,8.0,NaN,NaN
9,975900,/m/03vyhn,Dos,NaN,M,NaN,NaN,Lobo Sebastian,NaN,/m/0bgchsc,/m/0bgchsg,/m/0bgchsp,2001.0,8.0,NaN,NaN


In [209]:
budget_raw = pd.read_csv(
    DATA_FOLDER + "Budget/movies_metadata.csv.zip",
    compression="zip",
)

In [210]:
budget = budget_raw[
    [
        "budget",
        "imdb_id",
        "original_title",
        "popularity",
        "revenue",
        "vote_average",
        "vote_count",
        "release_date",
    ]
].copy()

In [211]:
budget.columns = [
    "budget",
    "tconst",
    "name",
    "popularity",
    "revenue",
    "voteAverage",
    "voteCount",
    "releaseDate",
]

In [212]:
#budget["releaseMonth"] = pd.to_datetime(budget["releaseDate"], errors="coerce").dt.month
#budget["releaseYear"] = pd.to_datetime(budget["releaseDate"], errors="coerce").dt.year
#budget.drop_duplicates(subset=["name", "releaseMonth", "releaseYear"], inplace=True)
budget.drop(columns=["releaseDate", "name", "revenue", "voteAverage", "voteCount"], inplace=True)

In [213]:
budget["budget"] = pd.to_numeric(budget["budget"], errors="coerce")

In [214]:
# remove tconst nan entries to avoid issues when merging
print(budget.tconst.isnull().sum())
budget.dropna(subset='tconst', inplace=True)
print(budget.shape)

17
(45449, 3)


In [215]:
# remove duplicate tconst entries
print(budget.tconst.duplicated().sum())
budget.drop_duplicates(subset="tconst", inplace=True)
print(budget.shape)

32
(45417, 3)


In [216]:
# replace all zero budget entries to nan
print((budget['budget'] == 0).sum())
budget.loc[budget['budget'] == 0, 'budget'] = np.nan

36538


In [217]:
budget.head(10)

,budget,tconst,popularity
0,30000000.0,tt0114709,21.946943
1,65000000.0,tt0113497,17.015539
2,NaN,tt0113228,11.7129
3,16000000.0,tt0114885,3.859495
4,NaN,tt0113041,8.387519
5,60000000.0,tt0113277,17.924927
6,58000000.0,tt0114319,6.677277
7,NaN,tt0112302,2.561161
8,35000000.0,tt0114576,5.23158
9,58000000.0,tt0113189,14.686036


In [218]:
movies_imdb = pd.merge(movies_imdb, budget, on='tconst', how='left')
movies_imdb.shape

(81630, 20)

In [219]:
movies_imdb.head(100)

,wikiID,fbID,name,boxOffice,runtime,languages,countries,genresCmu,releaseMonth,releaseYear,tconst,titleType,originalTitle,isAdult,endYear,genresImdb,averageRating,numVotes,budget,popularity
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",2.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28463795,/m/0crgdbh,Brun bitter,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",NaN,1988.0,tt0094806,movie,Brun bitter,0,\N,"[Crime, Drama]",5.6,40.0,NaN,NaN
2,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",NaN,1987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,[German Language],[Germany],[Drama],NaN,1983.0,tt0083949,movie,Die flambierte Frau,0,\N,[Drama],6.0,621.0,NaN,NaN
4,13696889,/m/03cfc81,The Gangsters,NaN,35.0,"[Silent film, English Language]",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi...",5.0,1913.0,tt0002894,short,The Gangsters,0,\N,"[Comedy, Short]",6.8,16.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,24039457,/m/0crx1wl,ZA: Zombies Anonymous,NaN,118.0,[English Language],[United States of America],"[Parody, B-movie, Horror, Zombie Film]",NaN,2005.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,31322464,/m/0gj8s31,Youth Sports Festival,NaN,NaN,[German Language],[German Democratic Republic],"[Sports, Documentary]",NaN,1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,5894429,/m/0fc8w8,Closing the Ring,NaN,117.0,[English Language],"[United States of America, Canada, United King...","[Romantic drama, Romance Film, Drama, War film]",9.0,2007.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,12788657,/m/02x4zpv,The Boston Strangler,17810894.0,116.0,[English Language],[United States of America],"[Crime Fiction, Mystery, Thriller, Drama, Crim...",NaN,1968.0,tt0062755,movie,The Boston Strangler,0,\N,"[Crime, Drama, Mystery]",7.1,10502.0,NaN,3.877109
